## Notebook to extract and save 4DHumans keypoints (2D + 3D) as pickle file

In [ ]:
import numpy as np
import os
import joblib
import pickle

In [ ]:
def project(joints_3d, cam_trans, size):
    """
    Takes 3D keypoints and return 2D image coordinates
    """
    f = 5000
    K = np.array([[f/256, 0, 0],
                [0, f/256, 0],
                [0, 0, 1]])
    R = np.eye(3)

    points = joints_3d + cam_trans # (4,3)

    proj = (K @ R @ points.T).T # (4,3)
    proj_norm = proj[:,:2] / proj[:,2].reshape(-1,1)
    proj_im = proj_norm * max(size)
    proj_im += np.array(size[::-1])/2
    return proj_im


In [ ]:
path = '/home/lea/trampo/4DHumans/outputs/results' # Path to model outputs .pkl

for file in sorted(os.listdir(path)):
    all_keypoints = {}
    keypoint_pkl_path = None

    results = joblib.load(os.path.join(path, file))

    # Output setup
    json_output_dir = os.path.join('Keypoints', file.replace('demo_', '').split('.')[0] + '_json')
    pkl_output_dir = 'Keypoints'

    # Loop on all frames
    for frame in results: #['outputs//_DEMO/1_partie_0429-Camera1_M11139/img/000032.jpg']
        frame_res = results[frame]
        if len(frame_res['smpl']) == 0:
            continue

        keypoints = []
        keypoints_per_frame = []
        # Loop on detected ids
        for i, output in enumerate(frame_res['smpl']):
            try:
                id = frame_res['tracked_ids'][i]
            except IndexError:
                id = -1
            
            bbox = frame_res['bbox'][i]
            conf = frame_res['conf'][i]
            conf = np.ones((45,1)) * conf

            size = frame_res['size'][i]
            cam_trans = frame_res['camera'][i]

            joints_3d = frame_res['3d_joints'][i]           # Original 3D keypoints
            pts2d = project(joints_3d, cam_trans, size)     # Projected 3D keypoints -> 2D keypoints

            keypoints = np.hstack((pts2d, conf))            # Stack confidence score as ViTPose outputs

            keypoints_per_frame.append({"id": id,
                                        "bbox": bbox,
                                        "2d_keypoints": keypoints,
                                        "3d_keypoints": joints_3d})

        image_file = frame.split('/')[-3]
        frame_idx = frame.split('/')[-1].split('.')[0]

        all_keypoints[frame_idx] = keypoints_per_frame

        json_file_path = os.path.join(json_output_dir, f"{os.path.splitext(os.path.basename(image_file))[0]}_{frame_idx}.json")
        keypoint_pkl_path = os.path.join(pkl_output_dir, f"{os.path.splitext(os.path.basename(image_file))[0]}.pkl")

    # Save all keypoints as .pkl
    if not os.path.isdir(pkl_output_dir): os.makedirs(pkl_output_dir)
    if keypoint_pkl_path is not None and f"{os.path.splitext(os.path.basename(image_file))[0]}.pkl" not in os.listdir(pkl_output_dir):
        with open(keypoint_pkl_path, 'wb') as f:
            pickle.dump(all_keypoints, f)